# 05 — Análisis de Paper Trading

Este notebook analiza los resultados del **paper trading** (trading simulado con datos reales
del mercado en tiempo real). El paper trading es la fase final de validación antes
de operar con capital real.

1. Resumen de operaciones en paper trading
2. Evolución temporal del rendimiento
3. Análisis de position sizing adaptativo (Kelly Criterion)
4. Duración de trades y eficiencia
5. Comparación de modelos

> **Nota**: Se muestran métricas agregadas. Los detalles de position sizing son confidenciales.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from datetime import timedelta

sns.set_theme(style='whitegrid', palette='muted')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['figure.dpi'] = 100

DATA_DIR = Path('../data/sample')

In [ ]:
# Cargar trades del paper trading
trades = pd.read_csv(DATA_DIR / 'trades' / 'paper_trades.csv')
trades['entry_dt'] = pd.to_datetime(trades['entry_time'])
trades['exit_dt'] = pd.to_datetime(trades['exit_time'])
trades['duration'] = trades['exit_dt'] - trades['entry_dt']
trades['duration_hours'] = trades['duration'].dt.total_seconds() / 3600
trades = trades.sort_values('exit_dt').reset_index(drop=True)

print(f'Trades cargados: {len(trades)}')
print(f'Periodo: {trades["entry_dt"].min().date()} → {trades["exit_dt"].max().date()}')
print(f'Pares activos: {sorted(trades["symbol"].unique())}')
print(f'Direcciones: {sorted(trades["direction"].unique())}')

## 1. Resumen general del paper trading

El paper trading ejecuta las mismas señales y modelos que el sistema real,
pero sin capital real. Permite validar:
- Que las señales generan trades en condiciones reales de mercado
- Que los modelos RL toman decisiones coherentes
- Que la latencia y slippage no invalidan la estrategia

In [ ]:
# Resumen por par
def pair_summary(group):
    w = group[group['pnl_pips'] > 0]
    l = group[group['pnl_pips'] <= 0]
    gp = w['pnl_pips'].sum()
    gl = abs(l['pnl_pips'].sum())
    return pd.Series({
        'Trades': len(group),
        'Ganadores': len(w),
        'Perdedores': len(l),
        'WR (%)': len(w)/len(group)*100 if len(group) > 0 else 0,
        'PF': gp/gl if gl > 0 else np.inf,
        'P&L (pips)': group['pnl_pips'].sum(),
        'Duración media (h)': group['duration_hours'].mean()
    })

summary = trades.groupby('symbol').apply(pair_summary).round(2)
summary

## 2. Evolución temporal: curva de equity y rolling metrics

In [ ]:
# Equity curve
trades['cum_pnl'] = trades['pnl_pips'].cumsum()
trades['peak'] = trades['cum_pnl'].cummax()
trades['drawdown'] = trades['cum_pnl'] - trades['peak']

fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True,
                         gridspec_kw={'height_ratios': [3, 1, 1]})

# Equity
axes[0].plot(trades['exit_dt'], trades['cum_pnl'], color='steelblue', linewidth=1.5)
axes[0].fill_between(trades['exit_dt'], trades['cum_pnl'], alpha=0.1, color='steelblue')
axes[0].axhline(0, color='gray', linestyle='--', alpha=0.5)
axes[0].set_title('Paper Trading — Curva de Equity', fontsize=13)
axes[0].set_ylabel('P&L acumulado (pips)')

# Drawdown
axes[1].fill_between(trades['exit_dt'], trades['drawdown'], 0, color='crimson', alpha=0.4)
axes[1].set_ylabel('Drawdown (pips)')
axes[1].set_title(f'Drawdown (max: {trades["drawdown"].min():.1f} pips)', fontsize=11)

# Rolling Win Rate (últimos 20 trades)
rolling_wr = (trades['pnl_pips'] > 0).rolling(20).mean() * 100
axes[2].plot(trades['exit_dt'], rolling_wr, color='purple', linewidth=1.2)
axes[2].axhline(50, color='red', linestyle='--', alpha=0.5, label='50%')
axes[2].set_ylabel('WR (%)')
axes[2].set_title('Rolling Win Rate (ventana 20 trades)', fontsize=11)
axes[2].set_ylim(0, 100)
axes[2].legend()

plt.tight_layout()
plt.show()

## 3. Position sizing adaptativo

### Kelly Criterion

El sistema utiliza **Kelly Criterion** para ajustar el tamaño de posición de forma adaptativa.
El concepto se basa en maximizar el crecimiento logarítmico del capital:

- Señales con mejor historial → posiciones más grandes
- Señales nuevas o con pocos trades → posiciones conservadoras
- Rachas perdedoras → reducción automática

Esto contrasta con el sizing fijo (mismo tamaño para todos los trades),
que no diferencia entre señales de alta y baja confianza.

In [ ]:
# Análisis de tamaños de posición
if 'size' in trades.columns:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Distribución de tamaños
    axes[0].hist(trades['size'], bins=30, color='steelblue', alpha=0.7, edgecolor='white')
    axes[0].axvline(trades['size'].mean(), color='red', linestyle='--',
                    label=f'Media: {trades["size"].mean():.3f}')
    axes[0].set_xlabel('Tamaño de posición (lotes)', fontsize=11)
    axes[0].set_ylabel('Frecuencia', fontsize=11)
    axes[0].set_title('Distribución de Tamaños de Posición', fontsize=12)
    axes[0].legend()

    # Tamaño vs P&L
    colors = ['green' if x > 0 else 'red' for x in trades['pnl_pips']]
    axes[1].scatter(trades['size'], trades['pnl_pips'], c=colors, alpha=0.5, s=20)
    axes[1].axhline(0, color='gray', linestyle='--', alpha=0.5)
    axes[1].set_xlabel('Tamaño (lotes)', fontsize=11)
    axes[1].set_ylabel('P&L (pips)', fontsize=11)
    axes[1].set_title('Tamaño de Posición vs P&L', fontsize=12)

    plt.tight_layout()
    plt.show()

    print(f'Tamaño medio:   {trades["size"].mean():.3f} lotes')
    print(f'Tamaño mínimo:  {trades["size"].min():.3f} lotes')
    print(f'Tamaño máximo:  {trades["size"].max():.3f} lotes')
    print(f'Desv. estándar: {trades["size"].std():.3f} lotes')
else:
    print('Columna "size" no disponible en los datos sample.')

## 4. Duración de trades y eficiencia

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Distribución de duración
durations = trades['duration_hours'].dropna()
durations_capped = durations[durations < durations.quantile(0.95)]  # Excluir outliers

axes[0].hist(durations_capped, bins=40, color='steelblue', alpha=0.7, edgecolor='white')
axes[0].axvline(durations_capped.median(), color='red', linestyle='--',
                label=f'Mediana: {durations_capped.median():.1f}h')
axes[0].set_xlabel('Duración (horas)', fontsize=11)
axes[0].set_ylabel('Frecuencia', fontsize=11)
axes[0].set_title('Distribución de Duración de Trades', fontsize=12)
axes[0].legend()

# Eficiencia: P&L por hora
trades['pips_per_hour'] = trades['pnl_pips'] / trades['duration_hours'].replace(0, np.nan)
eff = trades['pips_per_hour'].dropna()
eff_capped = eff[(eff > eff.quantile(0.02)) & (eff < eff.quantile(0.98))]

axes[1].hist(eff_capped, bins=40, color='darkorange', alpha=0.7, edgecolor='white')
axes[1].axvline(0, color='red', linestyle='--', alpha=0.5)
axes[1].set_xlabel('Pips / hora', fontsize=11)
axes[1].set_ylabel('Frecuencia', fontsize=11)
axes[1].set_title('Eficiencia: P&L por Hora de Exposición', fontsize=12)

plt.tight_layout()
plt.show()

print(f'Duración media:   {durations.mean():.1f} horas')
print(f'Duración mediana:  {durations.median():.1f} horas')
print(f'Eficiencia media: {eff.mean():.2f} pips/hora')

## 5. Análisis por razón de salida

Las posiciones pueden cerrarse por distintas razones:
- **model_exit**: El modelo RL decide cerrar la posición
- **sl_hit**: Se alcanza el Stop Loss
- **tp_hit**: Se alcanza el Take Profit

In [ ]:
if 'exit_reason' in trades.columns:
    exit_analysis = trades.groupby('exit_reason').agg(
        Trades=('pnl_pips', 'count'),
        WR=('pnl_pips', lambda x: (x > 0).mean() * 100),
        PnL_medio=('pnl_pips', 'mean'),
        PnL_total=('pnl_pips', 'sum'),
        Duracion_media_h=('duration_hours', 'mean')
    ).round(2)

    print('=== Análisis por Razón de Salida ===')
    display(exit_analysis) if hasattr(__builtins__, '__IPYTHON__') else print(exit_analysis)

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Trades por razón
    exit_analysis['Trades'].plot(kind='bar', ax=axes[0], color='steelblue', alpha=0.7)
    axes[0].set_title('Trades por Razón de Salida', fontsize=12)
    axes[0].set_ylabel('Cantidad')
    axes[0].tick_params(axis='x', rotation=45)

    # P&L medio por razón
    colors = ['green' if x > 0 else 'red' for x in exit_analysis['PnL_medio']]
    exit_analysis['PnL_medio'].plot(kind='bar', ax=axes[1], color=colors, alpha=0.7)
    axes[1].axhline(0, color='gray', linestyle='--')
    axes[1].set_title('P&L Medio por Razón de Salida (pips)', fontsize=12)
    axes[1].set_ylabel('Pips')
    axes[1].tick_params(axis='x', rotation=45)

    plt.tight_layout()
    plt.show()

## 6. Evolución semanal

In [ ]:
# P&L semanal
trades['week'] = trades['exit_dt'].dt.isocalendar().week.astype(int)
trades['year_week'] = trades['exit_dt'].dt.strftime('%Y-W%U')

weekly = trades.groupby('year_week').agg(
    trades=('pnl_pips', 'count'),
    pnl=('pnl_pips', 'sum'),
    wr=('pnl_pips', lambda x: (x > 0).mean() * 100)
).reset_index()

fig, ax = plt.subplots(figsize=(14, 5))
colors = ['green' if x > 0 else 'red' for x in weekly['pnl']]
ax.bar(range(len(weekly)), weekly['pnl'], color=colors, alpha=0.7)
ax.axhline(0, color='gray', linewidth=0.5)
ax.set_xticks(range(len(weekly)))
ax.set_xticklabels(weekly['year_week'], rotation=45, ha='right', fontsize=8)
ax.set_title('P&L Semanal (pips)', fontsize=13)
ax.set_ylabel('P&L (pips)')

# Anotar número de trades por semana
for i, row in weekly.iterrows():
    ax.text(i, row['pnl'] + (2 if row['pnl'] >= 0 else -4),
            f"{row['trades']}t", ha='center', fontsize=7, color='gray')

plt.tight_layout()
plt.show()

print(f'Semanas positivas: {(weekly["pnl"] > 0).sum()}/{len(weekly)} '
      f'({(weekly["pnl"] > 0).mean()*100:.0f}%)')
print(f'P&L semanal medio: {weekly["pnl"].mean():.1f} pips')

## 7. Conclusiones del paper trading

### Validación en condiciones reales

El paper trading confirma (o no) que los resultados del backtesting se mantienen
cuando el sistema opera con datos de mercado en tiempo real:

- **Latencia**: Las señales se ejecutan con delay real (~5 min tras cierre de vela)
- **Slippage**: Se incluye spread real del broker
- **Disponibilidad**: El sistema opera 24/5 de forma autónoma

### Position sizing adaptativo

El uso de Kelly Criterion permite que el sistema ajuste automáticamente
el tamaño de posición según la confianza acumulada en cada señal:
- Señales nuevas empiezan con tamaños conservadores
- A medida que acumulan historial, el tamaño se adapta
- Las rachas perdedoras reducen el tamaño automáticamente

### Pipeline completo

```
Descubrimiento → Optuna → Training → Test → Paper Trading → [Producción]
    (Phase 1)    (Phase 3) (Phase 4) (Phase 5)   (validación)
```

Solo las señales que superan **todas** las fases, incluyendo paper trading,
se consideran válidas para operar con capital real.

---

*Fin de la serie de notebooks. Para más detalles sobre la metodología,
consultar la documentación en `docs/`.*